In [1]:
import numpy as np

In [13]:
def determinant(A: np.ndarray):
    # A 为n*n的方阵
    if A.shape[0] == 1:
        return A[0, 0]
    if A.shape[0] == 2:
        return A[0, 0] * A[1, 1] - A[0, 1] * A[1, 0]
    
    det = 0
    for j in range(A.shape[0]):
        sub_matrix = np.concatenate((A[1:, :j], A[1:, j+1:]), axis=1)
        det += (-1)**j * A[0, j] * determinant(sub_matrix)
    return det

def QR_decomposition(A: np.ndarray):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((m, n))
    
    for k in range(n):
        v = A[:, k]
        for j in range(k):
            R[j, k] = Q[:, j] @ A[:, k]
            v = v - R[j, k] * Q[:, j]
            
        R[k, k] = np.linalg.norm(v)
        Q[:, k] = v / R[k, k]
        
    return Q, R


def QR_decomposition2(A):
    m, n = A.shape
    list_beta = []
    Q, B = [], []
    K = np.eye(n)
    
    for i in range(n):
        beta = A[:, i]
        for j in range(i):
            k_ij = (A[:, i].dot(list_beta[j])) / (list_beta[j].dot(list_beta[j]))
            K[j, i] = k_ij
            beta -= k_ij * list_beta[j]
        list_beta.append(beta)
        beta_norm = np.linalg.norm(beta)
        B.append(beta_norm)
        Q.append(beta / beta_norm)
        
    Q = np.array(Q).T
    B = B * np.identity(n)
    R = B @ K
    
    return Q, R        
            

def eigen_value(A: np.ndarray, max_iter=1000, tol=1e-8):
    # A 为n*n的方阵
    k = 0
    Ak = A.copy()
    Q_total = np.identity(A.shape[0])
    
    while k < max_iter:
        Q, R = QR_decomposition(Ak)
        Ak = R @ Q
        Q_total = Q_total @ Q
        
        # 判断收敛：是否接近上三角矩阵
        off = np.linalg.norm(Ak - np.triu(Ak))
        if off < tol:
            break
        k += 1
        
    eigvals = np.diag(Ak)
    
    return eigvals

def eigen_vector(A, eigvals, max_iter=10000, tol=1e-6):
    n = A.shape[0]
    alpha = 1e-3
    vectors = []
    
    for _lambda in eigvals:
        v = np.random.rand(n)
        v /= np.linalg.norm(v)
        
        for i in range(max_iter):
            grad = (A.T @ A - 2 * _lambda * A + _lambda ** 2 * np.identity(n)) @ v
            v -= alpha * grad
            v /= np.linalg.norm(v)
            
            # 判断收敛
            if np.linalg.norm(grad) < tol:
                break
            
        vectors.append(v)
    
    return np.array(vectors).T  # 每列是一个特征向量

In [16]:
A = np.array([
    [1., 2., 2.],
    [2., 1., 2.],
    [1., 2., 1.]
])

Q, R = QR_decomposition2(A)

print(Q @ R)

[[1. 2. 2.]
 [2. 1. 2.]
 [1. 2. 1.]]
